# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'11-20-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'11-20-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-11-21 05:25:51,33.93911,67.709953,44443,1661,35370,7412.0,Afghanistan,114.166180,3.737371
1,NaN,NaN,NaN,Albania,2020-11-21 05:25:51,41.15330,20.168300,31459,672,15055,15732.0,Albania,1093.161443,2.136114
2,NaN,NaN,NaN,Algeria,2020-11-21 05:25:51,28.03390,1.659600,72755,2236,47581,22938.0,Algeria,165.913956,3.073328
3,NaN,NaN,NaN,Andorra,2020-11-21 05:25:51,42.50630,1.521800,6142,76,5239,827.0,Andorra,7949.265515,1.237382
4,NaN,NaN,NaN,Angola,2020-11-21 05:25:51,-11.20270,17.873900,14267,334,7117,6816.0,Angola,43.409249,2.341067


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,42609,42795,42969,43035,43240,43403,43628,43851,44228,44443
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,25801,26211,26701,27233,27830,28432,29126,29837,30623,31459
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,64257,65108,65975,66819,67679,68589,69591,70629,71652,72755


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1581,1591,1595,1605,1617,1626,1638,1645,1650,1661
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,590,598,605,612,623,631,637,646,657,672
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2093,2111,2124,2139,2154,2168,2186,2206,2224,2236


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,34967,35024,35036,35067,35092,35137,35160,35295,35350,35370
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,12493,12574,12667,12767,12889,13453,13804,14216,14565,15055
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,42980,42980,43779,44199,44633,45148,45148,46326,46962,47581


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2351,2385,2417,2435,2456,2481,2506,2529,2554,2580
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,7409,7454,7523,7596,7646,7696,7772,7849,7933,8038
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1112,1113,1117,1123,1128,1130,1134,1137,1145,1151


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,32,32,36,36,36,36,37,37,39,39
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,84,84,84,84,84,84,84,84,84,84
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,9,9,9,9,9,9,9,10,10,10


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
646,1001.0,Autauga,Alabama,US,2020-11-21 05:25:51,32.539527,-86.644082,2580,39,0,2541.0,"Autauga, Alabama, US",4617.945551,1.511628
683,1075.0,Lamar,Alabama,US,2020-11-21 05:25:51,33.779950,-88.096680,601,7,0,594.0,"Lamar, Alabama, US",4353.495110,1.164725
684,1077.0,Lauderdale,Alabama,US,2020-11-21 05:25:51,34.901719,-87.656247,3653,53,0,3600.0,"Lauderdale, Alabama, US",3939.436422,1.450862


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,11910858,254413,4457930
India,9050597,132726,8478124
Brazil,6020164,168613,5430896


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,11910858,254413,4457930,7198515,2020-11-21 05:25:51,37.936303,-91.379001
India,9050597,132726,8478124,439747,2020-11-21 05:25:51,23.088275,81.806127
Brazil,6020164,168613,5430896,420655,2020-11-21 05:25:51,-12.669522,-48.480493
France,2160343,48341,155829,1956173,2020-11-21 05:25:51,8.314863,-21.695174
Russia,2023025,34980,1539352,448693,2020-11-21 05:25:51,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
Texas,1117495,20751,0
California,1089750,18640,0
Florida,923418,17889,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
Texas,1117495,20751,0,1096744,2020-11-21 05:25:51,31.660643,-98.653069
California,1089750,18640,0,1071110,2020-11-21 05:25:51,37.843962,-120.728594
Florida,923418,17889,0,905529,2020-11-21 05:25:51,28.940755,-82.700744
Illinois,634395,11795,0,622600,2020-11-21 05:25:51,39.843618,-89.178330
New York,584850,34252,0,550598,2020-11-21 05:25:51,42.544151,-75.474183


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,357451,7396,0
Illinois,Cook,272733,6201,0
Florida,Miami-Dade,211257,3746,0
Arizona,Maricopa,185580,3878,0
Texas,Harris,179911,2954,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,357451,7396,0,350055,2020-11-21 05:25:51,34.308284,-118.228241,6037.0
Illinois,Cook,272733,6201,0,266532,2020-11-21 05:25:51,41.841448,-87.816588,17031.0
Florida,Miami-Dade,211257,3746,0,207511,2020-11-21 05:25:51,25.611236,-80.551706,12086.0
Arizona,Maricopa,185580,3878,0,181702,2020-11-21 05:25:51,33.348359,-112.491815,4013.0
Texas,Harris,179911,2954,0,176957,2020-11-21 05:25:51,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-18,43851,29837,70629,6018,13922,139,1339337,120459,27785,221688,...,4296,70858,1,98350,1300,66186,10,2083,17280,8981
2020-11-19,44228,30623,71652,6066,14134,139,1349434,121979,27792,228683,...,4377,70921,1,98665,1304,67296,10,2086,17350,9046
2020-11-20,44443,31459,72755,6142,14267,139,1359042,123646,27806,235351,...,4477,71208,1,99017,1305,68768,10,2090,17373,9120


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-18,1645,646,2206,76,332,4,36347,1839,907,2054,...,68,601,0,860,35,589,1,607,356,261
2020-11-19,1650,657,2224,76,333,4,36532,1870,907,2116,...,69,601,0,863,35,598,1,608,356,265
2020-11-20,1661,672,2236,76,334,4,36790,1900,907,2224,...,69,603,0,866,35,606,1,608,356,265


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-18,35295,14216,46326,5055,6623,127,1156474,80069,25507,142738,...,3404,68123,0,93390,1124,56843,8,1417,16495,8184
2020-11-19,35350,14565,46962,5127,7062,127,1167514,88388,25511,149541,...,3479,68123,0,93691,1142,57469,8,1428,16591,8195
2020-11-20,35370,15055,47581,5239,7117,127,1177819,90754,25521,156483,...,3547,68451,0,94046,1142,57879,8,1434,16606,8229


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2351,2385,2417,2435,2456,2481,2506,2529,2554,2580
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,570,576,579,581,584,584,588,596,600,601
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,3088,3140,3221,3259,3298,3360,3431,3487,3578,3653
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,1028,1037,1050,1071,1076,1080,1098,1124,1152,1179
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,6994,7043,7122,7181,7212,7240,7278,7331,7363,7421


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-18,223486,25588,0,283102,137617,1064040,176694,97028,29755,49,...,325201,1091558,162028,3161,1463,208833,135424,36266,350665,25275
2020-11-19,225910,26081,0,287225,139855,1077174,182801,99381,30153,49,...,328088,1105009,165996,3310,1469,210787,137411,37399,358113,26169
2020-11-20,228373,26544,0,291696,141916,1089750,188566,101469,30816,49,...,331532,1117495,170584,3459,1482,213331,139543,38480,365190,27129


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2020-11-18        2529    7849    1137  1008   2648     678   1102    5764   
2020-11-19        2554    7933    1145  1011   2683     680   1113    5814   
2020-11-20        2580    8038    1151  1024   2704     684   1120    5896   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-18         1620      935  ...    1119    279     1456      311   
2020-11-19         1641      956  ...    1139    280     1486      335   
2020-11-20         1663      959  ...    1158    282     1592      349   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-18           1224  1271   835          0      263    353  
2020-11-19           1269  1298   869          0      274    355  
2020-11-20           1353  1336   894          0      290    364  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-18,3347,100,0,6365,2275,18453,2651,4784,742,0,...,4048,20338,740,60,23,3860,2592,612,2925,155
2020-11-19,3419,101,0,6384,2297,18542,2730,4805,742,0,...,4128,20565,756,61,23,3896,2603,623,3010,176
2020-11-20,3451,101,0,6427,2321,18640,2745,4828,746,0,...,4202,20751,773,62,23,3912,2619,639,3091,176


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-11-18          37      84      10   18     35      19     41      95   
2020-11-19          39      84      10   18     35      19     41     111   
2020-11-20          39      84      10   18     35      19     41     113   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-18           48       16  ...       4      5        9        1   
2020-11-19           48       24  ...       4      5       10        1   
2020-11-20           48       24  ...       4      5       10        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-18              4     2     4          0        7      0  
2020-11-19              6     2     4          0        7      0  
2020-11-20              6     2     4          0        7      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-18,0.005111,0.024411,0.014916,0.011259,0.007526,0.037313,0.007774,0.013368,0.000288,0.033043,...,0.020913,0.002972,0.0,0.003060,0.009317,0.019265,0.0,0.000961,0.002146,0.004025
2020-11-19,0.008597,0.026343,0.014484,0.007976,0.015228,0.000000,0.007539,0.012618,0.000252,0.031553,...,0.018855,0.000889,0.0,0.003203,0.003077,0.016771,0.0,0.001440,0.004051,0.007238
2020-11-20,0.004861,0.027300,0.015394,0.012529,0.009410,0.000000,0.007120,0.013666,0.000504,0.029158,...,0.022847,0.004047,0.0,0.003568,0.000767,0.021874,0.0,0.001918,0.001326,0.008180


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-18,0.004274,0.014129,0.009149,0.0,0.012195,0.0,0.006675,0.015461,0.0,0.056041,...,0.000000,0.003339,NaN,0.002331,0.0,0.015517,0.0,0.000000,0.008499,0.003846
2020-11-19,0.003040,0.017028,0.008160,0.0,0.003012,0.0,0.005090,0.016857,0.0,0.030185,...,0.014706,0.000000,NaN,0.003488,0.0,0.015280,0.0,0.001647,0.000000,0.015326
2020-11-20,0.006667,0.022831,0.005396,0.0,0.003003,0.0,0.007062,0.016043,0.0,0.051040,...,0.000000,0.003328,NaN,0.003476,0.0,0.013378,0.0,0.000000,0.000000,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-18,0.003840,0.029846,0.026092,0.018127,0.006229,0.0,0.006651,0.022031,0.000078,0.045799,...,0.016726,0.002885,NaN,0.003287,0.000000,0.020814,0.0,0.004965,0.001336,0.002450
2020-11-19,0.001558,0.024550,0.013729,0.014243,0.066284,0.0,0.009546,0.103898,0.000157,0.047661,...,0.022033,0.000000,NaN,0.003223,0.016014,0.011013,0.0,0.007763,0.005820,0.001344
2020-11-20,0.000566,0.033642,0.013181,0.021845,0.007788,0.0,0.008826,0.026768,0.000392,0.046422,...,0.019546,0.004815,NaN,0.003789,0.000000,0.007134,0.0,0.004202,0.000904,0.004149


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-18,0.011945,0.021844,NaN,0.011454,0.012619,0.011171,0.027028,0.021498,0.006869,0.0,...,0.013943,0.011756,0.019320,0.018363,0.008966,0.010016,0.009715,0.026667,0.024872,0.033616
2020-11-19,0.010846,0.019267,NaN,0.014564,0.016263,0.012344,0.034563,0.024251,0.013376,0.0,...,0.008878,0.012323,0.024490,0.047137,0.004101,0.009357,0.014672,0.031241,0.021240,0.035371
2020-11-20,0.010903,0.017752,NaN,0.015566,0.014737,0.011675,0.031537,0.021010,0.021988,0.0,...,0.010497,0.011299,0.027639,0.045015,0.008850,0.012069,0.015515,0.028905,0.019762,0.036685


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-18      0.009178  0.009907  0.002646  0.003984  0.020817  0.000000   
2020-11-19      0.009885  0.010702  0.007036  0.002976  0.013218  0.002950   
2020-11-20      0.010180  0.013236  0.005240  0.012859  0.007827  0.005882   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-11-18      0.002730  0.010873  0.015674  0.017410  ...  0.024725   
2020-11-19      0.009982  0.008675  0.012963  0.022460  ...  0.017873   
2020-11-20      0.006289  0.014104  0.013406  0.003138  ...  0.016681   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-11-18      0.014545  0.033357  0.047138   0.063423  0.015176  0.048995   
2020-11-19      0.003584  0.020604  0.077170   0.036765  0.021243  0.040719   
2020-11-20      0.007143  0.071332  0.041791   0.066194  0.029276  0.028769   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-11-18            NaN  0.039526  0.020231  
2020-11-19            NaN  0.041825  0.005666  
2020-11-20            NaN  0.058394  0.025352  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-18,0.013935,0.00,NaN,0.008397,0.013363,0.005778,0.016488,0.002725,0.004060,NaN,...,0.013267,0.009480,0.010929,0.016949,0.0,0.006519,0.008168,0.023411,0.02023,0.000000
2020-11-19,0.021512,0.01,NaN,0.002985,0.009670,0.004823,0.029800,0.004390,0.000000,NaN,...,0.019763,0.011161,0.021622,0.016667,0.0,0.009326,0.004244,0.017974,0.02906,0.135484
2020-11-20,0.009359,0.00,NaN,0.006736,0.010448,0.005285,0.005495,0.004787,0.005391,NaN,...,0.017926,0.009044,0.022487,0.016393,0.0,0.004107,0.006147,0.025682,0.02691,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                       \
Admin2           Autauga Baldwin   Barbour      Bibb    Blount Bullock Butler   
2020-11-18      0.000000     0.0  0.111111  0.058824  0.029412     0.0    0.0   
2020-11-19      0.054054     0.0  0.000000  0.000000  0.000000     0.0    0.0   
2020-11-20      0.000000     0.0  0.000000  0.000000  0.000000     0.0    0.0   

Province_State                               ... Wyoming                   \
Admin2           Calhoun Chambers  Cherokee  ...    Park Platte  Sheridan   
2020-11-18      0.233766      0.0  0.066667  ...     0.0    0.0  0.000000   
2020-11-19      0.168421      0.0  0.500000  ...     0.0    0.0  0.111111   
2020-11-20      0.018018      0.0  0.000000  ...     0.0    0.0  0.000000   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-18          0.0        0.0   0.0   0.0        NaN      0.0    NaN  
2020-11-19          0.0        0.5   0.0   0.0        NaN      0.0    NaN  
2020-11-20          0.0        0.0   0.0   0.0        NaN      0.0    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-18,0.004794,0.023585,0.014400,0.010170,0.010082,0.019133,0.007406,0.011064,0.000479,0.031067,...,0.021550,0.003104,0.0,0.003357,0.006989,0.017034,7.784992e-47,0.001212,0.002517,0.004898
2020-11-19,0.006695,0.024964,0.014442,0.009073,0.012655,0.009566,0.007472,0.011841,0.000365,0.031310,...,0.020202,0.001997,0.0,0.003280,0.005033,0.016903,3.892496e-47,0.001326,0.003284,0.006068
2020-11-20,0.005778,0.026132,0.014918,0.010801,0.011032,0.004783,0.007296,0.012754,0.000435,0.030234,...,0.021525,0.003022,0.0,0.003424,0.002900,0.019388,1.946248e-47,0.001622,0.002305,0.007124


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-18,0.005454,0.012951,0.008342,0.000833,0.010692,0.010417,0.007726,0.014930,2.637492e-10,0.045263,...,0.008874,0.002591,NaN,0.003119,2.631776e-25,0.014487,0.0,0.000832,0.004791,0.004982
2020-11-19,0.004247,0.014989,0.008251,0.000417,0.006852,0.005208,0.006408,0.015893,1.318746e-10,0.037724,...,0.011790,0.001295,NaN,0.003304,1.315888e-25,0.014884,0.0,0.001239,0.002395,0.010154
2020-11-20,0.005457,0.018910,0.006823,0.000208,0.004928,0.002604,0.006735,0.015968,6.593731e-11,0.044382,...,0.005895,0.002312,NaN,0.003390,6.579441e-26,0.014131,0.0,0.000620,0.001198,0.005077


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-18,0.002356,0.028013,0.015751,0.019841,0.008456,0.000829,0.007554,0.023077,0.000262,0.046517,...,0.017087,0.003187,NaN,0.003607,0.002610,0.016367,4.561518e-49,0.004283,0.003890,0.003118
2020-11-19,0.001957,0.026281,0.014740,0.017042,0.037370,0.000414,0.008550,0.063487,0.000209,0.047089,...,0.019560,0.001593,NaN,0.003415,0.009312,0.013690,2.280759e-49,0.006023,0.004855,0.002231
2020-11-20,0.001262,0.029962,0.013960,0.019444,0.022579,0.000207,0.008688,0.045128,0.000301,0.046755,...,0.019553,0.003204,NaN,0.003602,0.004656,0.010412,1.140380e-49,0.005112,0.002880,0.003190


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-18,0.009874,0.024195,NaN,0.010323,0.011784,0.010403,0.027139,0.022624,0.009887,1.505266e-75,...,0.013669,0.010576,0.019204,0.025537,0.007608,0.010082,0.013434,0.026113,0.022462,0.038154
2020-11-19,0.010360,0.021731,NaN,0.012443,0.014023,0.011373,0.030851,0.023437,0.011631,7.526329e-76,...,0.011273,0.011449,0.021847,0.036337,0.005855,0.009720,0.014053,0.028677,0.021851,0.036762
2020-11-20,0.010631,0.019742,NaN,0.014005,0.014380,0.011524,0.031194,0.022224,0.016810,3.763165e-76,...,0.010885,0.011374,0.024743,0.040676,0.007352,0.010894,0.014784,0.028791,0.020806,0.036723


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-18      0.009532  0.009196  0.002967  0.006543  0.015262  0.000772   
2020-11-19      0.009708  0.009949  0.005002  0.004760  0.014240  0.001861   
2020-11-20      0.009944  0.011593  0.005121  0.008809  0.011033  0.003872   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-11-18      0.004203  0.010507  0.012809  0.012576  ...  0.018805   
2020-11-19      0.007093  0.009591  0.012886  0.017518  ...  0.018339   
2020-11-20      0.006691  0.011847  0.013146  0.010328  ...  0.017510   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-11-18      0.028792  0.042528  0.049695   0.050994  0.025232  0.045290   
2020-11-19      0.016188  0.031566  0.063433   0.043879  0.023237  0.043004   
2020-11-20      0.011666  0.051449  0.052612   0.055037  0.026257  0.035886   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-11-18           -1.0  0.057392  0.017367  
2020-11-19           -1.0  0.049609  0.011516  
2020-11-20           -1.0  0.054001  0.018434  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-18,0.011437,0.005943,NaN,0.004923,0.012332,0.004077,0.013865,0.002644,0.003285,NaN,...,0.013104,0.007059,0.011053,0.008479,2.113773e-06,0.005668,0.007960,0.019755,0.021362,0.023528
2020-11-19,0.016474,0.007972,NaN,0.003954,0.011001,0.004450,0.021833,0.003517,0.001643,NaN,...,0.016434,0.009110,0.016338,0.012573,1.056887e-06,0.007497,0.006102,0.018864,0.025211,0.079506
2020-11-20,0.012917,0.003986,NaN,0.005345,0.010725,0.004868,0.013664,0.004152,0.003517,NaN,...,0.017180,0.009077,0.019412,0.014483,5.284433e-07,0.005802,0.006124,0.022273,0.026061,0.039753


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-18      0.009154  0.000035  0.055556  0.029550  0.015913  0.000332   
2020-11-19      0.031604  0.000017  0.027778  0.014775  0.007956  0.000166   
2020-11-20      0.015802  0.000009  0.013889  0.007388  0.003978  0.000083   

Province_State                                                  ...   Wyoming  \
Admin2                Butler   Calhoun      Chambers  Cherokee  ...      Park   
2020-11-18      2.980232e-09  0.117432  6.577846e-07  0.033333  ...  0.010417   
2020-11-19      1.490116e-09  0.142927  3.288923e-07  0.266667  ...  0.005208   
2020-11-20      7.450581e-10  0.080472  1.644462e-07  0.133333  ...  0.002604   

Province_State                                                              \
Admin2            Platte  Sheridan Sublette Sweetwater     Teton     Uinta   
2020-11-18      0.007830  0.031418      0.0   0.000031  0.000015  0.000163   
2020-11-19      0.003915  0.071264      0.0   0.250015  0.000008  0.000081   
2020-11-20      0.001958  0.035632      0.0   0.125008  0.000004  0.000041   

Province_State                                     
Admin2         Unassigned      Washakie    Weston  
2020-11-18      -0.972376  1.212660e-12 -0.533333  
2020-11-19      -0.972376  6.063298e-13 -0.533333  
2020-11-20      -0.972376  3.031649e-13 -0.533333  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201120,AK,25369,NaN,879343,NaN,totalTestsViral,904712,127.0,606.0,...,904712,0,13,69903ec40b2eb2629d77fa81ecaea343e862a768,0,0,0,0,0,NaN
1,20201120,AL,228373,36965.0,1316804,NaN,totalTestsViral,1508212,1329.0,23449.0,...,1545177,32,154,f80f4da63db3493044b482c48fbd160331904b94,0,0,0,0,0,NaN
2,20201120,AR,141916,16133.0,1440090,NaN,totalTestsViral,1565873,928.0,8353.0,...,1582006,24,85,d592ed990843af9225a3b264b6846c263bf770b1,0,0,0,0,0,NaN
3,20201120,AS,0,NaN,1988,NaN,totalTestsViral,1988,NaN,NaN,...,1988,0,0,75b3e9d8e73bac0c045fc4f011d5272817c31ae9,0,0,0,0,0,NaN
4,20201120,AZ,291696,8299.0,1780492,NaN,totalTestsPeopleViral,2063889,1835.0,24052.0,...,2072188,43,181,1dae08a03c619044abeec56918bc249ec753c326,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201120,AK,25369.0,NaN,879343.0,NaN,totalTestsViral,904712.0,127.0,606.0,...,904712,0,13,69903ec40b2eb2629d77fa81ecaea343e862a768,0,0,0,0,0,NaN
1,20201120,AL,228373.0,36965.0,1316804.0,NaN,totalTestsViral,1508212.0,1329.0,23449.0,...,1545177,32,154,f80f4da63db3493044b482c48fbd160331904b94,0,0,0,0,0,NaN
2,20201120,AR,141916.0,16133.0,1440090.0,NaN,totalTestsViral,1565873.0,928.0,8353.0,...,1582006,24,85,d592ed990843af9225a3b264b6846c263bf770b1,0,0,0,0,0,NaN
3,20201120,AS,0.0,NaN,1988.0,NaN,totalTestsViral,1988.0,NaN,NaN,...,1988,0,0,75b3e9d8e73bac0c045fc4f011d5272817c31ae9,0,0,0,0,0,NaN
4,20201120,AZ,291696.0,8299.0,1780492.0,NaN,totalTestsPeopleViral,2063889.0,1835.0,24052.0,...,2072188,43,181,1dae08a03c619044abeec56918bc249ec753c326,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-11-20,AK,25369,NaN,879343,NaN,totalTestsViral,904712,127.0,606.0,NaN,...,904712,0,13,69903ec40b2eb2629d77fa81ecaea343e862a768,0,0,0,0,0,NaN
2020-11-20,AL,228373,36965.0,1316804,NaN,totalTestsViral,1508212,1329.0,23449.0,NaN,...,1545177,32,154,f80f4da63db3493044b482c48fbd160331904b94,0,0,0,0,0,NaN
2020-11-20,AR,141916,16133.0,1440090,NaN,totalTestsViral,1565873,928.0,8353.0,359.0,...,1582006,24,85,d592ed990843af9225a3b264b6846c263bf770b1,0,0,0,0,0,NaN
2020-11-20,AS,0,NaN,1988,NaN,totalTestsViral,1988,NaN,NaN,NaN,...,1988,0,0,75b3e9d8e73bac0c045fc4f011d5272817c31ae9,0,0,0,0,0,NaN
2020-11-20,AZ,291696,8299.0,1780492,NaN,totalTestsPeopleViral,2063889,1835.0,24052.0,431.0,...,2072188,43,181,1dae08a03c619044abeec56918bc249ec753c326,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-11-20,AK,25369.0,NaN,879343.0,NaN,totalTestsViral,904712.0,127.0,606.0,NaN,...,904712,0,13,69903ec40b2eb2629d77fa81ecaea343e862a768,0,0,0,0,0,NaN
2020-11-20,AL,228373.0,36965.0,1316804.0,NaN,totalTestsViral,1508212.0,1329.0,23449.0,NaN,...,1545177,32,154,f80f4da63db3493044b482c48fbd160331904b94,0,0,0,0,0,NaN
2020-11-20,AR,141916.0,16133.0,1440090.0,NaN,totalTestsViral,1565873.0,928.0,8353.0,359.0,...,1582006,24,85,d592ed990843af9225a3b264b6846c263bf770b1,0,0,0,0,0,NaN
2020-11-20,AS,0.0,NaN,1988.0,NaN,totalTestsViral,1988.0,NaN,NaN,NaN,...,1988,0,0,75b3e9d8e73bac0c045fc4f011d5272817c31ae9,0,0,0,0,0,NaN
2020-11-20,AZ,291696.0,8299.0,1780492.0,NaN,totalTestsPeopleViral,2063889.0,1835.0,24052.0,431.0,...,2072188,43,181,1dae08a03c619044abeec56918bc249ec753c326,0,0,0,0,0,NaN
